Processing to do
---
- Nombre moyen de points par saison puit retirer la colonne point
- Vitesse de pointe moyenne par saison 
- Nombre de courses terminee par saison
- le circuit ayant le Nombre max de victoire 
- compter le nombre de fois premier
- compter le nombre de fois dans le top 10

In [36]:
%matplotlib inline
import pandas as pd
import numpy as np

## 1. Loading the Data sets

In [37]:
df_drivers = pd.read_csv("../data/inputs/drivers.csv")
df_races_results = pd.read_csv("../data/outputs/02_races_results.csv")

In [38]:
df_drivers.drop(['driverRef', 'url'], axis=1, inplace=True)

In [39]:
df_drivers.head()

,driverId,number,code,forename,surname,dob,nationality
0,1,44,HAM,Lewis,Hamilton,1985-01-07,British
1,2,\N,HEI,Nick,Heidfeld,1977-05-10,German
2,3,6,ROS,Nico,Rosberg,1985-06-27,German
3,4,14,ALO,Fernando,Alonso,1981-07-29,Spanish
4,5,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish


In [40]:
df_races_results.head()

,resultId,raceId,driverId,constructorId,grid,points,laps,milliseconds,fastestLap,race_rank,fastestLapTime,fastestLapSpeed,statusId,year,round,circuitId,name
0,1,18,1,1,1,10.0,58,5690616,39,2,87452,218.300,1,2008,1,1,Australian Grand Prix
1,2,18,2,2,5,8.0,58,5696094,41,3,87739,217.586,1,2008,1,1,Australian Grand Prix
2,3,18,3,3,7,6.0,58,5698779,41,5,88090,216.719,1,2008,1,1,Australian Grand Prix
3,4,18,4,4,11,5.0,58,5707797,58,7,88603,215.464,1,2008,1,1,Australian Grand Prix
4,5,18,5,1,3,4.0,58,5708630,43,1,87418,218.385,1,2008,1,1,Australian Grand Prix


## 2. Add a flag for actives drivers

In [41]:
df_drivers['driver_is_active'] = 0

In [42]:
list_drivers = [
    ("Max", "Verstappen"),
    ("Logan", "Sargeant"),
    ("Lando", "Norris"),
    ("Pierre", "Gasly"),
    ("Sergio", "Pérez"),
    ("Fernando", "Alonso"),
    ("Charles", "Leclerc"),
    ("Lance", "Stroll"),
    ("Kevin", "Magnussen"),
    ("Yuki", "Tsunoda"),
    ("Alexander", "Albon"),
    ("Esteban", "Ocon"),
    ("Lewis", "Hamilton"),
    ("Carlos", "Sainz"),
    ("George", "Russell"),
    ("Oscar", "Piastri"),
    ("Guanyu", "Zhou"),
    ("Nico", "Hülkenberg"),
    ("Valtteri", "Bottas")
]

In [43]:
df_drivers.head()

,driverId,number,code,forename,surname,dob,nationality,driver_is_active
0,1,44,HAM,Lewis,Hamilton,1985-01-07,British,0
1,2,\N,HEI,Nick,Heidfeld,1977-05-10,German,0
2,3,6,ROS,Nico,Rosberg,1985-06-27,German,0
3,4,14,ALO,Fernando,Alonso,1981-07-29,Spanish,0
4,5,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,0


In [44]:
for (name, surname) in list_drivers:
    df_drivers.loc[(df_drivers['forename'] == name) & (df_drivers['surname'] == surname), 'driver_is_active'] = 1

df_drivers[df_drivers.driver_is_active == 1].head()

,driverId,number,code,forename,surname,dob,nationality,driver_is_active
0,1,44,HAM,Lewis,Hamilton,1985-01-07,British,1
3,4,14,ALO,Fernando,Alonso,1981-07-29,Spanish,1
452,842,10,GAS,Pierre,Gasly,1996-02-07,French,1
807,807,27,HUL,Nico,Hülkenberg,1987-08-19,German,1
814,815,11,PER,Sergio,Pérez,1990-01-26,Mexican,1


## 3. Add the calculated features

### 3.1. Average of points by driver

In [45]:
ids = df_drivers.driverId.to_list()

In [46]:
df_drivers['driver_avg_point'] = np.float64(0)

for driverId in ids:
    avg_points = df_races_results[df_races_results.driverId == driverId]['points'].mean()
    df_drivers.loc[df_drivers.driverId == driverId, 'driver_avg_point'] = np.float64(avg_points)

In [47]:
df_drivers.head()

,driverId,number,code,forename,surname,dob,nationality,driver_is_active,driver_avg_point
0,1,44,HAM,Lewis,Hamilton,1985-01-07,British,1,14.100932
1,2,\N,HEI,Nick,Heidfeld,1977-05-10,German,0,1.407609
2,3,6,ROS,Nico,Rosberg,1985-06-27,German,0,7.740291
3,4,14,ALO,Fernando,Alonso,1981-07-29,Spanish,1,5.954054
4,5,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,0,0.937500


### 3.2. Average speed by driver

In [48]:
df_drivers['driver_avg_speed'] = np.float64(0)

for driverId in ids:
    avg_speed = df_races_results[(df_races_results.driverId == driverId) & (df_races_results.fastestLapSpeed > 0)]['fastestLapSpeed'].mean()
    df_drivers.loc[df_drivers.driverId == driverId, 'driver_avg_speed'] = np.float64(avg_speed)

In [49]:
df_drivers.fillna({'driver_avg_speed': 0}, inplace=True)

In [50]:
# Find the avg speed of Max Verstappen
df_drivers[df_drivers.forename.str.contains('Max')].head()

,driverId,number,code,forename,surname,dob,nationality,driver_is_active,driver_avg_point,driver_avg_speed
348,349,\N,\N,Max,Jean,1943-07-27,French,0,0.000000,0.000000
720,720,\N,\N,Max,de Terra,1918-10-06,Swiss,0,0.000000,0.000000
819,820,4,CHI,Max,Chilton,1991-04-21,British,0,0.000000,193.597971
829,830,33,VER,Max,Verstappen,1997-09-30,Dutch,1,13.002857,208.454187


### 3.3 Number of races ends by seaoson (From 2015)

In [51]:
years_list = [0, 2019, 2020, 2021, 2022, 2023]
df_drivers['race_end_bf_2019'] = 0
df_drivers['race_end_in_2019'] = 0
df_drivers['race_end_in_2020'] = 0
df_drivers['race_end_in_2021'] = 0
df_drivers['race_end_in_2022'] = 0
df_drivers['race_end_in_2023'] = 0

for driverId in ids:
    for year in years_list:
        if 0 == year:
            nber_of_win = df_races_results[(df_races_results.driverId == driverId) & (df_races_results.statusId == 1) & (df_races_results.year < 2019)].raceId.count()
            df_drivers.loc[df_drivers.driverId == driverId, 'race_end_bf_2019'] = nber_of_win
        else:
            nber_of_win = df_races_results[(df_races_results.driverId == driverId) & (df_races_results.statusId == 1) & (df_races_results.year == year)].raceId.count()
            df_drivers.loc[df_drivers.driverId == driverId, f'race_end_in_{year}'] = nber_of_win

In [52]:
df_drivers.head()

,driverId,number,code,forename,surname,dob,nationality,driver_is_active,driver_avg_point,driver_avg_speed,race_end_bf_2019,race_end_in_2019,race_end_in_2020,race_end_in_2021,race_end_in_2022,race_end_in_2023
0,1,44,HAM,Lewis,Hamilton,1985-01-07,British,1,14.100932,205.711759,193,21,16,21,19,12
1,2,\N,HEI,Nick,Heidfeld,1977-05-10,German,0,1.407609,204.044578,66,0,0,0,0,0
2,3,6,ROS,Nico,Rosberg,1985-06-27,German,0,7.740291,200.290369,145,0,0,0,0,0
3,4,14,ALO,Fernando,Alonso,1981-07-29,Spanish,1,5.954054,204.462531,202,0,0,11,12,12
4,5,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,0,0.937500,198.246165,39,0,0,0,0,0


### 3.4. Most won circuits

In [53]:
df_drivers['driver_most_won_circuit_id'] = 0

for driverId in ids:
    try:
        df_drivers.loc[df_drivers.driverId == driverId, 'driver_most_won_circuit_id'] = df_races_results[
            (df_races_results.driverId == driverId) & (df_races_results.race_rank == 1)
                ].groupby(['circuitId', 'name'])[['resultId']].count().sort_values(
                    ascending=False, by='resultId').resultId.idxmax()[0]
    except:
        df_drivers.loc[df_drivers.driverId == driverId, 'driver_most_won_circuit_id'] = 0

In [54]:
df_drivers.head()

,driverId,number,code,forename,surname,dob,nationality,driver_is_active,driver_avg_point,driver_avg_speed,race_end_bf_2019,race_end_in_2019,race_end_in_2020,race_end_in_2021,race_end_in_2022,race_end_in_2023,driver_most_won_circuit_id
0,1,44,HAM,Lewis,Hamilton,1985-01-07,British,1,14.100932,205.711759,193,21,16,21,19,12,14
1,2,\N,HEI,Nick,Heidfeld,1977-05-10,German,0,1.407609,204.044578,66,0,0,0,0,0,2
2,3,6,ROS,Nico,Rosberg,1985-06-27,German,0,7.740291,200.290369,145,0,0,0,0,0,3
3,4,14,ALO,Fernando,Alonso,1981-07-29,Spanish,1,5.954054,204.462531,202,0,0,11,12,12,9
4,5,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,0,0.937500,198.246165,39,0,0,0,0,0,1


### 3.5. Number of races won

In [55]:
df_drivers['driver_nber_of_races_won'] = 0

for driverId in ids:
    df_drivers.loc[df_drivers.driverId == driverId, 'driver_nber_of_races_won'] = df_races_results[(df_races_results.driverId == driverId) & (df_races_results.race_rank == 1)].shape[0]

In [56]:
df_drivers.head()

,driverId,number,code,forename,surname,dob,nationality,driver_is_active,driver_avg_point,driver_avg_speed,race_end_bf_2019,race_end_in_2019,race_end_in_2020,race_end_in_2021,race_end_in_2022,race_end_in_2023,driver_most_won_circuit_id,driver_nber_of_races_won
0,1,44,HAM,Lewis,Hamilton,1985-01-07,British,1,14.100932,205.711759,193,21,16,21,19,12,14,63
1,2,\N,HEI,Nick,Heidfeld,1977-05-10,German,0,1.407609,204.044578,66,0,0,0,0,0,2,2
2,3,6,ROS,Nico,Rosberg,1985-06-27,German,0,7.740291,200.290369,145,0,0,0,0,0,3,20
3,4,14,ALO,Fernando,Alonso,1981-07-29,Spanish,1,5.954054,204.462531,202,0,0,11,12,12,9,22
4,5,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,0,0.937500,198.246165,39,0,0,0,0,0,1,2


### 3.6. Number of times in the top 10 table

In [57]:
df_drivers['driver_nber_of_times_in_top_10'] = 0

for driverId in ids:
    df_drivers.loc[df_drivers.driverId == driverId, 'driver_nber_of_times_in_top_10'] = df_races_results[(df_races_results.driverId == driverId) & (df_races_results.race_rank <= 10)].shape[0]

In [58]:
df_drivers.head()

,driverId,number,code,forename,surname,dob,nationality,driver_is_active,driver_avg_point,driver_avg_speed,race_end_bf_2019,race_end_in_2019,race_end_in_2020,race_end_in_2021,race_end_in_2022,race_end_in_2023,driver_most_won_circuit_id,driver_nber_of_races_won,driver_nber_of_times_in_top_10
0,1,44,HAM,Lewis,Hamilton,1985-01-07,British,1,14.100932,205.711759,193,21,16,21,19,12,14,63,292
1,2,\N,HEI,Nick,Heidfeld,1977-05-10,German,0,1.407609,204.044578,66,0,0,0,0,0,2,2,131
2,3,6,ROS,Nico,Rosberg,1985-06-27,German,0,7.740291,200.290369,145,0,0,0,0,0,3,20,158
3,4,14,ALO,Fernando,Alonso,1981-07-29,Spanish,1,5.954054,204.462531,202,0,0,11,12,12,9,22,285
4,5,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,0,0.937500,198.246165,39,0,0,0,0,0,1,2,37


In [59]:
df_drivers['dob'] = pd.to_datetime(df_drivers['dob'])

In [60]:
df_drivers['age'] = pd.to_datetime('now').year - df_drivers['dob'].dt.year

In [61]:
df_drivers.to_csv('../data/outputs/03_drivers_processed.csv', index=False)